# Scrape the table from the Wikipedia page

In [47]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib

html = urllib.request.urlopen("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").read()
soup = BeautifulSoup(html, 'html.parser')
table = soup.find('table', {'class': 'wikitable sortable'})
columns = [th.text.replace('\n', '') 
for th in table.find('tr').find_all('th')]
trs = table.find_all('tr')[1:]
rows = list()
for tr in trs:
    rows.append([td.text.replace('\n', '').replace('\xa0', '') for td in tr.find_all('td')])
df = pd.DataFrame(data=rows, columns=columns)
df.rename(columns = {'Postcode':'PostalCode'},inplace=True)
df.rename(columns = {'Neighbourhood':'Neighborhood'},inplace=True)
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


# Delete the rows whose 'Borough' is 'Not assigned'

In [48]:
df = df[~df['Borough'].str.contains('Not assigned')]
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


# Combine the rows whose 'Neighborhood' shares the same postcode

In [49]:
df = df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


# Change the 'Not assigned' in 'Neighborhood' with 'Borough' value

In [50]:
df.loc[df['Neighborhood']=='Not assigned','Neighborhood'] = df.loc[df['Neighborhood']=='Not assigned']['Borough']

In [51]:
df.loc[df['PostalCode']=='M7A']

,PostalCode,Borough,Neighborhood
85,M7A,Queen's Park,Queen's Park


# Done!

In [52]:
df.shape

(103, 3)